In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
class Riiid:
    
    path = None
    
    dtype={'row_id': 'int64', 'timestamp': 'int64',
           'user_id': 'int32', 'content_id': 'int16',
           'content_type_id': 'int8', 'task_container_id': 'int16',
           'user_answer': 'int8', 'answered_correctly': 'int8',
           'prior_question_elapsed_time': 'float32',
           'prior_question_had_explanation': 'boolean',
          }
    
    usecols=['timestamp', 'user_id', 'content_id',
             'content_type_id','task_container_id', 'answered_correctly',
             'prior_question_elapsed_time','prior_question_had_explanation']
    
    questions_df = None
    lectures_df = None
    train_user_target_stats = None
    train_question_target_stats = None
    train_user_question_types_sum = None
    train_user_lecture_types_sum = None   
    scaler = None # ['timestamp', 'task_container_id', 'prior_question_elapsed_time']

    def __init__(self):
        pass

    
    @staticmethod
    def _proc_questions(df):
        df.fillna('-1', inplace=True) # str(-1): str.split()
        
        # collect unique tags
        all_tags = []
        for tag in df['tags']:
            for t in tag.split():
                if int(t) not in all_tags:
                    all_tags.append(int(t))
        all_tags.sort()
        all_tags.remove(-1) # -1 means no tags
        
        # add tags as columns
        df = df.join(pd.DataFrame(columns=all_tags)).fillna(0)
        
        # set to 1 if a row contain the tag
        for row in range(len(df)):
            for t in df.at[row, 'tags'].split():
                if int(t) in df.columns:
                    df.at[row, int(t)] = 1
        
        return df
    
    @classmethod
    def load_and_process_questions(cls):
        
        cls.questions_df = pd.read_csv(cls.path/'questions.csv')
        
        # drop columns
        cls.questions_df = cls.questions_df.drop(columns=['correct_answer'])
        
        # add number of tags
        cls.questions_df['question_tag_count'] = cls.questions_df['tags'].map(lambda x: len(str(x).split()))
        
        
        # process questions data
        cls.questions_df = pd.get_dummies(cls.questions_df,
                                         prefix=['part'],
                                         columns=['part'])
        
        # add number of questions in bundle
        tmp = cls.questions_df[['question_id', 'bundle_id']] \
            .groupby('bundle_id').count() \
            .rename(columns={'question_id':'bundle_size'})
        
        cls.questions_df = cls.questions_df.merge(tmp, how='left', on='bundle_id')
        
        #cls.questions_df = cls._proc_questions(cls.questions_df)
        
    @classmethod
    def load_and_process_lectures(cls):
        
        cls.lectures_df = pd.read_csv(cls.path/'lectures.csv')
        

        # process lectures data
        cls.lectures_df = pd.get_dummies(cls.lectures_df,
                                         prefix=['type', 'part'],
                                         columns=['type_of', 'part'])        

    
    @classmethod
    def set_data_path(cls, path):
        cls.path = Path(path)
    
    @classmethod
    def _get_user_target_stats(cls, train):
        tmp=train[train['content_type_id']==0]
        
        # user part
        cls.train_user_target_stats = tmp[['user_id', 'answered_correctly']] \
            .groupby('user_id').agg(['mean', 'count', 'sum',]).reset_index()
        
        
        cls.train_user_target_stats.columns = ['user_id', 'user_mean',
                                               'user_count', 'user_sum',]
        
        cls.train_user_target_stats = cls.train_user_target_stats.astype(
            dtype = {'user_id':'int32', 'user_mean':'float32',
                     'user_count':'int32', 'user_sum':'int32'})

    @classmethod
    def _get_question_target_stats(cls, train):
        tmp=train[train['content_type_id']==0]

        # question part
        cls.train_question_target_stats = tmp[['content_id', 'answered_correctly']] \
            .groupby('content_id').agg(['mean', 'count', 'sum',]).reset_index()
        
        cls.train_question_target_stats.columns = ['question_id', 'question_mean',
                                                   'question_count', 'question_sum',]
        
        cls.train_question_target_stats = cls.train_question_target_stats.astype(
            dtype = {'question_id':'int16', 'question_mean':'float32',
                     'question_count':'int32', 'question_sum':'int32'})
        
    @classmethod
    def _get_question_stats(cls, train):
        tmp=train[train['content_type_id']==0]
        
        cls.train_user_question_types_sum = tmp[['user_id', 'content_id']] \
            .merge(cls.questions_df, how='left',
                   left_on='content_id', right_on='question_id') \
            .drop(columns=['content_id', 'question_id','bundle_id', 'tags', 'question_tag_count', 'bundle_size']) \
            .groupby('user_id').sum().reset_index()
        
        
        cls.train_user_question_types_sum.columns = ['user_id', 'q_part_1_sum',
                                                     'q_part_2_sum','q_part_3_sum',
                                                     'q_part_4_sum','q_part_5_sum',
                                                     'q_part_6_sum','q_part_7_sum']
        
        cls.train_user_question_types_sum = cls.train_user_question_types_sum.astype(
            dtype = {'user_id':'int32', 'q_part_1_sum':'int16',
                     'q_part_2_sum':'int16', 'q_part_3_sum':'int16',
                     'q_part_4_sum':'int16', 'q_part_5_sum':'int16',
                     'q_part_6_sum':'int16', 'q_part_7_sum':'int16'})
        
    @classmethod
    def _get_lecture_stats(cls, train):
        tmp=train[train['content_type_id']==1]
        
        cls.train_user_lecture_types_sum = tmp[['user_id', 'content_id']] \
            .merge(cls.lectures_df, how='left',
                   left_on='content_id', right_on='lecture_id') \
            .drop(columns=['content_id', 'lecture_id', 'tag']) \
            .groupby('user_id').sum().reset_index()
        
        
        cls.train_user_lecture_types_sum.columns = ['user_id', 'type_concept_sum',
                                                    'type_intention_sum', 'type_solving question_sum',
                                                    'type_starter_sum', 'l_part_1_sum',
                                                    'l_part_2_sum','l_part_3_sum','l_part_4_sum','l_part_5_sum',
                                                    'l_part_6_sum', 'l_part_7_sum']
        
        cls.train_user_lecture_types_sum = cls.train_user_lecture_types_sum.astype(
            dtype = {'user_id':'int32', 'type_concept_sum':'int16',
                     'type_intention_sum':'int16', 'type_solving question_sum':'int16',
                     'type_starter_sum':'int16', 'l_part_1_sum':'int16',
                     'l_part_2_sum':'int16', 'l_part_3_sum':'int16',
                     'l_part_4_sum':'int16', 'l_part_5_sum':'int16',
                     'l_part_6_sum':'int16', 'l_part_7_sum':'int16'})
     
    @classmethod
    def _get_scaler(cls, train):
        scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
        tmp = train[['timestamp', 'task_container_id', 'prior_question_elapsed_time']]
        cls.scaler =  scaler.fit(tmp.fillna(0.))

        
    def setup_data(self, df):
        
        if Riiid.train_user_target_stats is None:
            Riiid._get_user_target_stats(df)
        print('train_user_target_stats - Done')
        
        if Riiid.train_question_target_stats is None:
            Riiid._get_question_target_stats(df)
        print('train_question_target_stats - Done')
        
        Riiid._get_scaler(df)
        print('scaler - Done')
        
        if Riiid.train_user_question_types_sum is None:
            Riiid._get_question_stats(df)
        print('question_stats - Done')
        
        if Riiid.train_user_lecture_types_sum is None:
            Riiid._get_lecture_stats(df)
        print('lecture_stats - Done')
    
    def process_train_data(self, train, test=False):
        
        if not test: # we need questions and lectures for test
            # step 0 = keep questions only
            train = train[train['content_type_id']==0]
            print('step 0 (keep questions only) - Done')

        # step 1 = fillna for prior_question_elapsed_time and prior_question_had_explanation
        train = train.fillna({'prior_question_elapsed_time':0.,
                                    'prior_question_had_explanation':False})
        print('step 1 (fillna) - Done')
        
        # step 2 merge question without question_id, content_type_id and tags
        train = train.merge(self.questions_df, how='left',
                            left_on='content_id',right_on='question_id') \
                .drop(columns=['question_id','content_type_id', 'tags'])
        
        # fillna fillna mainly for lectures
        train = train.fillna({'prior_question_elapsed_time':0.,
                              'prior_question_had_explanation':False,
                              'bundle_id':0, 'bundle_size':0, 'question_tag_count':0, 'bundle_size':0})
        # change dtype
        train = train.astype({'bundle_id':'int16', 'bundle_size':'int16',
                              'question_tag_count':'int8', 'bundle_size':'int8'})
        
        print('step 2 (merge question) - Done')
               
        # step 3 merge question target stats
        train = train.merge(self.train_question_target_stats, how='left',
                            left_on='content_id', right_on='question_id') \
                .drop(columns=['content_id', 'question_id'])
        print('step 3 (merge question target) - Done')
        
        # step 4 merge train_user_target_stats
        train = train.merge(self.train_user_target_stats, how='left', on='user_id')
        print('step 4 (merge train_user_target) - Done')
        
        # step 5 merge train_user_question_types_sum
        train = train.merge(self.train_user_question_types_sum, how='left', on='user_id')
        print('step 5 (merge train_user_question_types) - Done')
        
        # step 6 merge train_user_lecture_types_sum and fillna with 0.
        # drop user_id
        train = train.merge(self.train_user_lecture_types_sum, how='left', on='user_id') \
            .fillna({ 'type_concept_sum':0.,'type_intention_sum':0.,
                     'type_solving question_sum':0., 'type_starter_sum':0.,
                     'l_part_1_sum':0., 'l_part_2_sum':0., 'l_part_3_sum':0.,
                     'l_part_4_sum':0., 'l_part_5_sum':0., 'l_part_6_sum':0.,
                     'l_part_7_sum':0.}) \
            .astype({ 'type_concept_sum':'int16','type_intention_sum':'int16',
                     'type_solving question_sum':'int16', 'type_starter_sum':'int16',
                     'l_part_1_sum':'int16', 'l_part_2_sum':'int16', 'l_part_3_sum':'int16',
                     'l_part_4_sum':'int16', 'l_part_5_sum':'int16', 'l_part_6_sum':'int16',
                     'l_part_7_sum':'int16'}) \
            .drop(columns=['user_id'])
        print('step 6 (merge train_user_lecture_types) - Done')
        
        # step 7 transform ['timestamp', 'task_container_id', 'prior_question_elapsed_time']
        cols_transform = ['timestamp', 'task_container_id', 'prior_question_elapsed_time']
        train[cols_transform] = self.scaler.transform(train[cols_transform])
        print('step 7 (transform) - Done')
        
        return train
        
    def split_data(self, df, n_iter=30):
        """Split into train and validation datasets.
        
        Returns: train and validation indices"""
        
        recent = pd.DataFrame(columns=['user_id', 'timestamp', 'user_id-timestamp'])
        df['user_id-timestamp'] = df['user_id'].astype(str) + '-' + df['timestamp'].astype(str)
        counter = 0
        while counter < n_iter:
            tmp = df[~df['user_id-timestamp'].isin(recent['user_id-timestamp'])][['user_id', 'timestamp']].groupby(['user_id']).max().reset_index()
            recent = recent.append(tmp, ignore_index=True)
            recent['user_id-timestamp'] = recent['user_id'].astype(str) + '-' + recent['timestamp'].astype(str)
            counter+=1
        train_idx = df[~df['user_id-timestamp'].isin(recent['user_id-timestamp'])].index
        val_idx = df[df['user_id-timestamp'].isin(recent['user_id-timestamp'])].index
        df.drop(columns=['user_id-timestamp'], inplace=True)
    
        return train_idx, val_idx
    
    
    def save_data(self, df, name):
        df.to_feather(self.path/(name + '.feather'))
        
    def load_data(self, name):
        return pd.read_feather(self.path/(name + '.feather'))
    

In [3]:
Riiid.set_data_path(path=r'./data')

In [4]:
Riiid.load_and_process_questions()

In [5]:
Riiid.load_and_process_lectures()

In [6]:
Riiid.questions_df.head(2)

,question_id,bundle_id,tags,question_tag_count,part_1,part_2,part_3,part_4,part_5,part_6,part_7,bundle_size
0,0,0,51 131 162 38,4,1,0,0,0,0,0,0,1
1,1,1,131 36 81,3,1,0,0,0,0,0,0,1


In [7]:
Riiid.lectures_df.head(2)

,lecture_id,tag,type_concept,type_intention,type_solving question,type_starter,part_1,part_2,part_3,part_4,part_5,part_6,part_7
0,89,159,1,0,0,0,0,0,0,0,1,0,0
1,100,70,1,0,0,0,1,0,0,0,0,0,0


In [8]:
r = Riiid()

In [9]:
# r.save_data(data_df, name='data')

In [10]:
# train_idx, val_idx = r.split_data(data_df)

In [11]:
# r.save_data(data_df.iloc[train_idx].reset_index(drop=True), name='train')
# r.save_data(data_df.iloc[val_idx].reset_index(drop=True), name='val')

In [9]:
# train_df = pd.read_csv(r.path/'train.csv', nrows=500000, dtype=r.dtype, usecols=r.usecols)
train_df = r.load_data('train')
val_df = r.load_data('val')

In [10]:
train_df.head(2)

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,1,NaN,<NA>
1,56943,115,5716,0,2,1,37000.0,False


In [11]:
val_df.head(2)

,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,302994,115,108,0,12,0,23000.0,False
1,328686,115,7900,0,13,1,21000.0,False


In [12]:
r.questions_df.head(2)

,question_id,bundle_id,tags,question_tag_count,part_1,part_2,part_3,part_4,part_5,part_6,part_7,bundle_size
0,0,0,51 131 162 38,4,1,0,0,0,0,0,0,1
1,1,1,131 36 81,3,1,0,0,0,0,0,0,1


In [13]:
r.lectures_df.head(2)

,lecture_id,tag,type_concept,type_intention,type_solving question,type_starter,part_1,part_2,part_3,part_4,part_5,part_6,part_7
0,89,159,1,0,0,0,0,0,0,0,1,0,0
1,100,70,1,0,0,0,1,0,0,0,0,0,0


In [14]:
# Riiid.train_user_target_stats = r.load_data('train_user_target_stats')
# Riiid.train_question_target_stats =r.load_data('train_question_target_stats')
# Riiid.train_user_question_types_sum = r.load_data('train_user_question_types_sum')
# Riiid.train_user_lecture_types_sum =r.load_data('train_user_lecture_types_sum')

In [15]:
# Riiid.train_user_target_stats = r.load_data('data_user_target_stats')
# Riiid.train_question_target_stats =r.load_data('data_question_target_stats')
# Riiid.train_user_question_types_sum = r.load_data('data_user_question_types_sum')
# Riiid.train_user_lecture_types_sum =r.load_data('data_user_lecture_types_sum')

In [16]:
r.setup_data(train_df)

train_user_target_stats - Done
train_question_target_stats - Done
scaler - Done
question_stats - Done
lecture_stats - Done


In [17]:
# r.save_data(r.train_user_target_stats, name='train_user_target_stats')
# r.save_data(r.train_question_target_stats, name='train_question_target_stats')
# r.save_data(r.train_user_question_types_sum, name='train_user_question_types_sum')
# r.save_data(r.train_user_lecture_types_sum, name='train_user_lecture_types_sum')

In [18]:
# r.save_data(r.train_user_target_stats, name='data_user_target_stats')
# r.save_data(r.train_question_target_stats, name='data_question_target_stats')
# r.save_data(r.train_user_question_types_sum, name='data_user_question_types_sum')
# r.save_data(r.train_user_lecture_types_sum, name='data_user_lecture_types_sum')

In [19]:
r.train_user_target_stats.head(2)

,user_id,user_mean,user_count,user_sum
0,115,0.833333,12,10
1,5382,0.701031,97,68


In [20]:
r.train_question_target_stats.head(2)

,question_id,question_mean,question_count,question_sum
0,0,0.913167,6357,5805
1,1,0.896999,6932,6218


In [21]:
r.train_user_question_types_sum.head(2)

,user_id,q_part_1_sum,q_part_2_sum,q_part_3_sum,q_part_4_sum,q_part_5_sum,q_part_6_sum,q_part_7_sum
0,115,10,0,0,0,2,0,0
1,5382,12,32,0,0,53,0,0


In [22]:
r.train_user_lecture_types_sum.head(2)

,user_id,type_concept_sum,type_intention_sum,type_solving question_sum,type_starter_sum,l_part_1_sum,l_part_2_sum,l_part_3_sum,l_part_4_sum,l_part_5_sum,l_part_6_sum,l_part_7_sum
0,5382,1,0,0,0,1,0,0,0,0,0,0
1,8623,2,0,0,0,2,0,0,0,0,0,0


In [23]:
train_df.shape, val_df.shape

((88777729, 8), (12452603, 8))

Transformation train

In [ ]:
train_df = r.process_train_data(train_df)
# del train_df
# train_df = r.load_data('train_transformed')

step 0 (keep questions only) - Done
step 1 (fillna) - Done
step 2 (merge question) - Done
step 3 (merge question target) - Done
step 4 (merge train_user_target) - Done
step 5 (merge train_user_question_types) - Done


In [ ]:
# r.save_data(train_df, 'data_transformed')
# r.save_data(train_df, 'train_transformed')

In [27]:
train_df.isna().sum().sum()

0

In [28]:
train_df.head()

,timestamp,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,question_tag_count,part_1,part_2,part_3,...,type_intention_sum,type_solving question_sum,type_starter_sum,l_part_1_sum,l_part_2_sum,l_part_3_sum,l_part_4_sum,l_part_5_sum,l_part_6_sum,l_part_7_sum
0,-0.692831,-0.708965,1,-1.232640,False,5692,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.692826,-0.708254,1,0.590887,False,5716,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.692821,-0.709676,1,1.478009,False,128,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.692820,-0.707543,1,-0.296234,False,7860,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.692819,-0.706831,1,-0.690510,False,7922,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
val_df = r.process_train_data(val_df, test=True)

step 1 (fillna) - Done
step 2 (merge question) - Done
step 3 (merge question target) - Done
step 4 (merge train_user_target) - Done
step 5 (merge train_user_question_types) - Done
step 6 (merge train_user_lecture_types) - Done
step 7 (transform) - Done


In [30]:
val_df.head()

,timestamp,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,question_tag_count,part_1,part_2,part_3,...,type_intention_sum,type_solving question_sum,type_starter_sum,l_part_1_sum,l_part_2_sum,l_part_3_sum,l_part_4_sum,l_part_5_sum,l_part_6_sum,l_part_7_sum
0,-0.692805,-0.701143,0,-0.099096,False,108,3,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,-0.692803,-0.700432,1,-0.197665,False,7900,3,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,-0.692801,-0.699721,1,-0.049811,False,7901,3,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,-0.692799,-0.699010,1,-0.148380,False,7971,3,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,-0.692797,-0.698299,1,-0.197665,False,25,3,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
val_df[val_df['question_mean'].isna() & (val_df['answered_correctly']!=-1)]

,timestamp,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,question_tag_count,part_1,part_2,part_3,...,type_intention_sum,type_solving question_sum,type_starter_sum,l_part_1_sum,l_part_2_sum,l_part_3_sum,l_part_4_sum,l_part_5_sum,l_part_6_sum,l_part_7_sum
11997832,-0.322958,0.290081,1,-0.062133,False,10005,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
11997833,-0.322958,0.290081,0,-0.062133,False,10005,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
11997834,-0.322958,0.290081,1,-0.062133,False,10005,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
11997835,-0.322958,0.290081,1,-0.062133,False,10005,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
val_df[(val_df['answered_correctly']!=-1)].isna().sum()

timestamp                               0
task_container_id                       0
answered_correctly                      0
prior_question_elapsed_time             0
prior_question_had_explanation          0
bundle_id                               0
question_tag_count                      0
part_1                                  0
part_2                                  0
part_3                                  0
part_4                                  0
part_5                                  0
part_6                                  0
part_7                                  0
bundle_size                             0
question_mean                           4
question_count                          4
question_sum                            4
user_mean                         4635171
user_count                        4635171
user_sum                          4635171
q_part_1_sum                      4635171
q_part_2_sum                      4635171
q_part_3_sum                      

In [33]:
r.train_question_target_stats

,question_id,question_mean,question_count,question_sum
0,0,0.913167,6357,5805
1,1,0.896999,6932,6218
2,2,0.559936,39701,22230
3,3,0.784089,20967,16440
4,4,0.631420,26792,16917
...,...,...,...,...
13514,13518,0.788060,670,528
13515,13519,0.586445,723,424
13516,13520,0.684685,666,456
13517,13521,0.818321,655,536


In [34]:
# tmp = val_df.merge(r.questions_df, how='left',
#                             left_on='content_id',right_on='question_id') \
#                 .drop(columns=['question_id','content_type_id', 'tags'])

In [35]:
# tmp[tmp['answered_correctly']!=-1].isna().sum()

In [36]:
# val_df.merge(r.questions_df, how='left',
#                             left_on='content_id',right_on='question_id') \
#                 .drop(columns=['question_id','content_type_id', 'tags']) \
#                 .astype({'bundle_id':'int16', 'bundle_size':'int16',
#                          'question_tag_count':'int8', 'bundle_size':'int8'})

In [37]:
# train_df[['timestamp', 'user_id']].groupby('user_id').max().join(
#     val_df[['timestamp', 'user_id']].groupby('user_id').min(), how='outer',lsuffix='_train', rsuffix='_val').isna().sum()

In [38]:
# df = r.load_data('train')

In [39]:
# df[df['content_type_id']==0][['content_id', 'answered_correctly']].groupby('content_id').agg(['mean', 'count', 'sum']).describe()